<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/machine-learning-bookcamp/3_churn_prediction_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Churn prediction project

Churn is when customers stop using the services of a company. Thus, churn prediction
is about identifying customers who are likely to cancel their contracts soon.

If the company can do that, it can offer discounts on these services in an effort to
keep the users.

Imagine that we are working at a telecom company that offers phone and internet
services, and we have a problem: some of our customers are churning. They no longer
are using our services and are going to a different provider. 

We would like to prevent
that from happening, so we develop a system for identifying these customers and offer
them an incentive to stay. 

We want to target them with promotional messages and give
them a discount. We also would like to understand why the model thinks our customers
churn, and for that, we need to be able to interpret the model’s predictions.

##Setup

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import display

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
# content/gdrive/My Drive/Kaggle is the path where kaggle.json is  present in the Google Drive
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/kaggle-keys"

In [4]:
%%shell

# download dataset from kaggle> URL: https://www.kaggle.com/blastchar/telco-customer-churn
kaggle datasets download -d blastchar/telco-customer-churn

unzip -qq telco-customer-churn.zip
rm -rf telco-customer-churn.zip

  0% 0.00/172k [00:00<?, ?B/s]
100% 172k/172k [00:00<00:00, 54.5MB/s]


##Dataset

In [5]:
# let’s read our dataset
data_df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
len(data_df)

7043

In [6]:
data_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
data_df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [8]:
data_df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

###Initial data preparation

In [9]:
# converting it to numbers with 'coerce'  for nonnumeric data (such as spaces),
total_charges = pd.to_numeric(data_df.TotalCharges, errors="coerce")

# confirm that data indeed contains nonnumeric characters
data_df[total_charges.isnull()][["customerID", "TotalCharges"]]

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [10]:
# so, let's set the missing values to zero
data_df["TotalCharges"] = pd.to_numeric(data_df.TotalCharges, errors="coerce")
data_df["TotalCharges"] = data_df["TotalCharges"].fillna(0)

In [11]:
# Let’s make the column names uniform by lowercasing everything and replacing spaces with underscores
data_df.columns = data_df.columns.str.lower().str.replace(" ", "_")
string_columns = list(data_df.dtypes[data_df.dtypes == "object"].index)

for col in string_columns:
  data_df[col] = data_df[col].str.lower().str.replace(" ", "_")

In [12]:
# let’s look at our target variable
data_df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [13]:
# converting to Boolean
(data_df.churn == "yes").head()

0    False
1    False
2     True
3    False
4     True
Name: churn, dtype: bool

In [14]:
# converting the Boolean to integer
(data_df.churn == "yes").astype(int).head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

In [15]:
# so, let’s convert the target variable to numbers
data_df.churn = (data_df.churn == "yes").astype(int)

data_df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


Let's split the dataset.

In [16]:
# split such that 80% of the data goes to the train set and the remaining 20% goes to the test set.
df_train_full, df_test = train_test_split(data_df, test_size=0.2, random_state=1)

In [17]:
df_train_full.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [18]:
# let's split it one more time into train and validation
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

In [19]:
# Takes the column with the target variable, churn, and saves it outside the dataframe
y_train = df_train.churn.values
y_val = df_val.churn.values

In [20]:
# Deletes the churn columns
del df_train["churn"]
del df_val["churn"]

###Exploratory data analysis

We should always check for any missing values in the dataset because many machine
learning models cannot easily deal with missing data.

In [21]:
# let’s perform any additional null handling
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [22]:
# let's check the distribution of values in the target variable
df_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

We know the absolute numbers, but let’s also check the proportion of churned
users among all customers. 

For that, we need to divide the number of customers who
churned by the total number of customers. We know that 1,521 of 5,634 churned, so the proportion is-

$$1521 / 5634 =  0.27$$

This gives us the proportion of churned users, or the probability that a customer will
churn. As we see in the training dataset, approximately `27%` of the customers stopped
using our services, and the rest remained as customers.

The proportion of churned users, or the probability of churning, has a special
name: churn rate.

In [23]:
# let's calculate the churn rate
global_mean = df_train_full.churn.mean()
round(global_mean, 3)

0.27

Our churn dataset is an example of a so-called imbalanced dataset.

We can clearly see that: the churn rate in our data is `.27`, which is a strong indicator of class imbalance.



In [24]:
# let's create two lists for categorical and numerical variables
categorical_cols = [
  'gender', 'seniorcitizen', 'partner', 'dependents',
  'phoneservice', 'multiplelines', 'internetservice',
  'onlinesecurity', 'onlinebackup', 'deviceprotection',
  'techsupport', 'streamingtv', 'streamingmovies',
  'contract', 'paperlessbilling', 'paymentmethod'
]

numerical_cols = ['tenure', 'monthlycharges', 'totalcharges']

In [25]:
# First, we can see how many unique values each variable has
df_train_full[categorical_cols].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

###Feature importance

Knowing how other variables affect the target variable, churn, is the key to understanding
the data and building a good model. This process is called feature importance
analysis.

We have two different kinds of features: categorical and numerical. Each kind has
different ways of measuring feature importance, so we will look at each separately.

####Churn rate

We can look at all the distinct values of a variable. Then, for each variable, there’s a
group of customers: all the customers who have this value. 

For each such group, we
can compute the churn rate, which is the group churn rate. 

When we have it, we can
compare it with the global churn rate — the churn rate calculated for all the observations
at once.

Let’s check first for the gender variable.



In [26]:
female_mean = df_train_full[df_train_full.gender == "female"].churn.mean()
print(f"gender == female: {round(female_mean, 3) * 100}")

male_mean = df_train_full[df_train_full.gender == "male"].churn.mean()
print(f"gender == male: {round(male_mean, 3) * 100}")

print(f"global mean: {round(global_mean, 3) * 100}")

gender == female: 27.700000000000003
gender == male: 26.3
global mean: 27.0


In [27]:
female_mean / global_mean

1.0253955354648652

In [28]:
male_mean / global_mean

0.9749802969838747

The difference between the group rates for both females
and males is quite small, which indicates that knowing the gender of the customer
doesn’t help us identify whether they will churn.

Now let’s take a look at another variable: partner.

In [29]:
partner_yes = df_train_full[df_train_full.partner == "yes"].churn.mean()
print(f"partner == yes: {round(partner_yes, 3) * 100}")

partner_no = df_train_full[df_train_full.partner == "no"].churn.mean()
print(f"partner == no: {round(partner_no, 3) * 100}")

print(f"global mean: {round(global_mean, 3) * 100}")

partner == yes: 20.5
partner == no: 33.0
global mean: 27.0


In [30]:
partner_yes / global_mean

0.7594724924338315

In [31]:
partner_no / global_mean

1.2216593879412643

As we see, the rates for those who have a partner are quite different from rates for
those who don’t: `20%` and `33%`, respectively. 

It means that clients with no partner are
more likely to churn than the ones with a partner.

####Risk ratio

In statistics, the ratio between probabilities
in different groups is called the risk ratio, where risk refers to the risk of having the effect. 

In our case, the effect is churn, so it’s the risk of churning:

`risk = group rate / global rate`

For gender == female, for example, the risk of churning is 1.02:

`risk = 27.7% / 27% = 1.02`

Risk is a number between zero and infinity. It has a nice interpretation that tells you
how likely the elements of the group are to have the effect (churn) compared with the
entire population.

Let’s calculate the risks for gender and partner.

In [32]:
global_mean = df_train_full.churn.mean()

df_group = df_train_full.groupby(by="gender").churn.agg(["mean"])  # calculate the AVG(churn) part
df_group["diff"] = df_group["mean"] - global_mean                  # Calculates the difference between group churn rate and global rate
df_group["risk"] = df_group["mean"] / global_mean                  # Calculates the ricsk between group churn rate and global rate

df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


Let’s now do that for all categorical variables.

In [33]:
for col in categorical_cols:
  df_group = df_train_full.groupby(by=col).churn.agg(["mean"])  # calculate the AVG(churn) part
  df_group["diff"] = df_group["mean"] - global_mean             # Calculates the difference between group churn rate and global rate
  df_group["risk"] = df_group["mean"] / global_mean             # Calculates the ricsk between group churn rate and global rate
  display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


This way, just by looking at the differences and the risks, we can identify the most discriminative
features: the features that are helpful for detecting churn. 

Thus, we
expect that these features will be useful for our future models.

####Mutual information

Higher values of mutual information mean a higher degree of dependence: if the
mutual information between a categorical variable and the target is high, this categorical
variable will be quite useful for predicting the target. 

On the other hand, if the
mutual information is low, the categorical variable and the target are independent,
and thus the variable will not be useful for predicting the target.

In [34]:
def calculate_mi(series):
  return mutual_info_score(series, df_train_full.churn)

In [35]:
# Applies the function to each categorical column of the dataset
df_mi = df_train_full[categorical_cols].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name="MI")

display(df_mi.head())
display(df_mi.tail())

,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923


,MI
partner,0.009968
seniorcitizen,0.009410
multiplelines,0.000857
phoneservice,0.000229
gender,0.000117


As we see, `contract`, `onlinesecurity`, and `techsupport` are among the most
important features.

Indeed, we’ve already noted that `contract` and
`techsupport` are quite informative. 

It’s also not surprising that `gender` is among the least important features, so we shouldn’t expect it to be useful for the model.

####Correlation coefficient

Mutual information is a way to quantify the degree of dependency between two categorical
variables, but it doesn’t work when one of the features is numerical, so we cannot
apply it to the three numerical variables that we have.

We can, however, measure the dependency between a binary target variable and a
numerical variable. We can pretend that the binary variable is numerical (containing
only the numbers zero and one) and then use the classical methods from statistics to
check for any dependency between these variables.

One such method is the correlation coefficient.It is a value from –1 to 1.

* Positive correlation means that when one variable goes up, the other variable tends to go up as well.
* Zero correlation means no relationship between two variables: they are completely independent.
* Negative correlation occurs when one variable goes up and the other goes
down.



In [36]:
df_train_full[numerical_cols].corrwith(df_train_full.churn).to_frame("correlation")

,correlation
tenure,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353


* The correlation between `tenure` and churn is `–0.35`: it has a negative sign, so the longer customers stay, the less often they tend to churn.
* `monthlycharges` has a positive coefficient of `0.19`, which means that customers who pay more tend to leave more often.
* `totalcharges` has a negative correlation, which makes sense: the longer people stay with the company, the more they have paid in total, so it’s less likely that they will leave.

In [37]:
df_train_full.groupby(by="churn")[numerical_cols].mean()

,tenure,monthlycharges,totalcharges
churn,,,
0,37.531972,61.176477,2548.021627
1,18.070348,74.521203,1545.689415


##Feature engineering

Before we proceed to training, however, we need to perform the feature engineering
step: transforming all categorical variables to numeric features.

Let's use one-hot encoding for categorical variables.

In [38]:
# convert dataframe to a list of dictionaries
train_dict = df_train[categorical_cols + numerical_cols].to_dict(orient="records")
train_dict[0]

{'gender': 'male',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'bank_transfer_(automatic)',
 'tenure': 71,
 'monthlycharges': 86.1,
 'totalcharges': 6045.9}

Now we can use DictVectorizer for converting the dictionaries to a matrix.

If a feature is categorical, it applies the one-hot encoding
scheme, but if a feature is numerical, it’s left intact.

In [39]:
# convert the list of dictionaries to matrix
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
x_train = dv.transform(train_dict)

print(f"Old shape: {df_train_full.shape}")
print(f"New shape: {x_train.shape}")
x_train[0]

Old shape: (5634, 21)
New shape: (3774, 45)


array([0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 8.6100e+01, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 7.1000e+01, 6.0459e+03])

In [40]:
# let's see the names of all these columns
dv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'dependents=no',
 'dependents=yes',
 'deviceprotection=no',
 'deviceprotection=no_internet_service',
 'deviceprotection=yes',
 'gender=female',
 'gender=male',
 'internetservice=dsl',
 'internetservice=fiber_optic',
 'internetservice=no',
 'monthlycharges',
 'multiplelines=no',
 'multiplelines=no_phone_service',
 'multiplelines=yes',
 'onlinebackup=no',
 'onlinebackup=no_internet_service',
 'onlinebackup=yes',
 'onlinesecurity=no',
 'onlinesecurity=no_internet_service',
 'onlinesecurity=yes',
 'paperlessbilling=no',
 'paperlessbilling=yes',
 'partner=no',
 'partner=yes',
 'paymentmethod=bank_transfer_(automatic)',
 'paymentmethod=credit_card_(automatic)',
 'paymentmethod=electronic_check',
 'paymentmethod=mailed_check',
 'phoneservice=no',
 'phoneservice=yes',
 'seniorcitizen',
 'streamingmovies=no',
 'streamingmovies=no_internet_service',
 'streamingmovies=yes',
 'streamingtv=no',
 'streamingtv=no_internet_servic

As we see, for each categorical feature it creates multiple columns for each of its distinct values.

Features such as `tenure` and `totalcharges` keep the original names because
they are numerical; therefore, `DictVectorizer` doesn’t change them.

##Classification model

The translation of the logistic regression formula to Python is almost identical to the
linear regression case, except that at the end, we apply the sigmoid function.

```python
def logistic_regression(xi):
  score = bias
  for j in range(n):
    score = score + xi[j] * w[j]
  prob = sigmoid(score)
  return prob
```

Let's use Scikit-learn logistic regression model.

In [41]:
model = LogisticRegression(solver="liblinear", random_state=1)
model.fit(x_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

Let’s see how well the model performs. 

In [42]:
# perform one-hot encoding in exactly the same way as during training
val_dict = df_val[categorical_cols + numerical_cols].to_dict(orient="records")
x_val = dv.transform(val_dict)

Now we are ready to put this matrix to the model.

In [43]:
y_pred = model.predict_proba(x_val)

In [44]:
y_pred

array([[0.76508784, 0.23491216],
       [0.73113015, 0.26886985],
       [0.68054704, 0.31945296],
       ...,
       [0.94274614, 0.05725386],
       [0.38476895, 0.61523105],
       [0.93872763, 0.06127237]])

Here, the first column of the array contains the probability that the target is negative(no churn), and the second column contains the probability that the target is positive(churn).

These columns convey the same information. We know the probability of churn —
it’s `p` — and the probability of not churning is always `1 – p`, so we don’t need both columns.

Thus, it’s enough to take only the second column of the prediction.

In [45]:
"""
: means select all the rows
1 means select only the column at index 1, and because the indexing starts at 0, it’s the second column
"""
y_pred = model.predict_proba(x_val)[:, 1]

In [46]:
y_pred[:10]

array([0.23491216, 0.26886985, 0.31945296, 0.36519001, 0.04552533,
       0.4401701 , 0.01825128, 0.11323952, 0.0060698 , 0.19301306])

To get the binary predictions, we take the probabilities and cut them above a certain threshold. 

If the probability for a customer is higher than this threshold, we predict
churn, otherwise, not churn.

In [47]:
# Let’s write the results to the churn array
churn = y_pred >= 0.5

In [48]:
churn[:10]

array([False, False, False, False, False, False, False, False, False,
       False])

In [49]:
#let's calculate accuracy by casting boolean array to integer array and then taking its mean
(y_val == churn).mean()

0.8016129032258065

The model predictions matched the actual value 80% of the time, or the model makes correct predictions in 80% of cases. 

This is what we call the accuracy of the model.

##Model interpretation

We know that the logistic regression model has two parameters that it learns from data:

* $w_0$ is the bias term
* $w=(w_1, w_2, ..., w_n)$ is the weights vector


In [50]:
# get the bias term
model.intercept_[0]

-0.12198896163060387

In [51]:
# get the rest of the weights
model.coef_[0]

array([ 5.63350419e-01, -8.59198889e-02, -5.99419491e-01, -3.02820218e-02,
       -9.17069398e-02,  9.99283340e-02, -1.15868311e-01, -1.06048985e-01,
       -2.73676855e-02, -9.46212762e-02, -3.23348965e-01,  3.17228314e-01,
       -1.15868311e-01,  7.84317450e-04, -1.68096778e-01,  1.27134010e-01,
       -8.10261944e-02,  1.35706167e-01, -1.15868311e-01, -1.41826818e-01,
        2.57861725e-01, -1.15868311e-01, -2.63982375e-01, -2.12615137e-01,
        9.06261753e-02, -4.80303106e-02, -7.39586510e-02, -2.66766308e-02,
       -1.36245780e-01,  1.74748490e-01, -1.33815041e-01,  1.27134010e-01,
       -2.49122972e-01,  2.97090535e-01, -8.48662760e-02, -1.15868311e-01,
        7.87456252e-02, -9.90818957e-02, -1.15868311e-01,  9.29612449e-02,
        1.78141046e-01, -1.15868311e-01, -1.84261697e-01, -6.94862417e-02,
        4.47681302e-04])

In [52]:
# let's see which feature is associated with each weight
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'contract=month-to-month': 0.563,
 'contract=one_year': -0.086,
 'contract=two_year': -0.599,
 'dependents=no': -0.03,
 'dependents=yes': -0.092,
 'deviceprotection=no': 0.1,
 'deviceprotection=no_internet_service': -0.116,
 'deviceprotection=yes': -0.106,
 'gender=female': -0.027,
 'gender=male': -0.095,
 'internetservice=dsl': -0.323,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.116,
 'monthlycharges': 0.001,
 'multiplelines=no': -0.168,
 'multiplelines=no_phone_service': 0.127,
 'multiplelines=yes': -0.081,
 'onlinebackup=no': 0.136,
 'onlinebackup=no_internet_service': -0.116,
 'onlinebackup=yes': -0.142,
 'onlinesecurity=no': 0.258,
 'onlinesecurity=no_internet_service': -0.116,
 'onlinesecurity=yes': -0.264,
 'paperlessbilling=no': -0.213,
 'paperlessbilling=yes': 0.091,
 'partner=no': -0.048,
 'partner=yes': -0.074,
 'paymentmethod=bank_transfer_(automatic)': -0.027,
 'paymentmethod=credit_card_(automatic)': -0.136,
 'paymentmethod=electronic_check': 0.175,


To understand how the model works, let’s consider what happens when we apply this model. 

To build the intuition, let’s train a simpler and smaller model that uses only
three variables.

In [53]:
subset = ["contract", "tenure", "totalcharges"]
train_dict_small = df_train[subset].to_dict(orient="records")
dv_small = DictVectorizer(sparse=False)
dv_small.fit(train_dict_small)

x_small_train = dv_small.transform(train_dict_small)

# Let’s see which features the small model will use
dv_small.get_feature_names()

['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'tenure',
 'totalcharges']

In [54]:
# Let’s train the small model on this set of features
model_small = LogisticRegression(solver="liblinear", random_state=1)
model_small.fit(x_small_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [55]:
# Let’s first check the bias term
model_small.intercept_[0]

-0.6387618613273348

In [56]:
# get the rest of the weights
model_small.coef_[0]

array([ 9.09810808e-01, -1.44126995e-01, -1.40444567e+00, -9.69580199e-02,
        8.46546042e-04])

In [60]:
# let's see which feature is associated with each weight
dict(zip(dv_small.get_feature_names(), model_small.coef_[0].round(3)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'contract=month-to-month': 0.91,
 'contract=one_year': -0.144,
 'contract=two_year': -1.404,
 'tenure': -0.097,
 'totalcharges': 0.001}

In [58]:
# let's validate this model
val_dict_small = df_val[subset].to_dict(orient="records")

x_small_val = dv_small.transform(val_dict_small)

In [59]:
y_pred_small = model_small.predict_proba(x_small_val)[:, 1]
print(y_pred_small)

[0.38167308 0.26380942 0.17153253 ... 0.13113125 0.54615263 0.03609456]


##Using the model

Now we can apply the model to customers
for scoring them. It’s quite easy.

First, we take a customer we want to score and put all the variable values in a dictionary:

In [61]:
customer = {
  'customerid': '8879-zkjof',
  'gender': 'female',
  'seniorcitizen': 0,
  'partner': 'no',
  'dependents': 'no',
  'tenure': 41,
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'dsl',
  'onlinesecurity': 'yes',
  'onlinebackup': 'no',
  'deviceprotection': 'yes',
  'techsupport': 'yes',
  'streamingtv': 'yes',
  'streamingmovies': 'yes',
  'contract': 'one_year',
  'paperlessbilling': 'yes',
  'paymentmethod': 'bank_transfer_(automatic)',
  'monthlycharges': 79.85,
  'totalcharges': 3320.75,
}

Now we can use our model to see whether this customer is going to churn.

In [64]:
# convert this dictionary to a matrix
x_test = dv.transform([customer])

# Now we take this matrix and put it into the trained model
model.predict_proba(x_test)

array([[0.92667889, 0.07332111]])

For each customer, it outputs two numbers,
which are the probability of staying with the company and the probability of churn.

Because there’s only one customer, we get a tiny NumPy array with one row and two
columns.

In [66]:
# select the probability of churn
model.predict_proba(x_test)[0, 1]

0.07332111085213508

In [67]:
# select the probability of staying
model.predict_proba(x_test)[0, 0]

0.9266788891478649

We see that the output is `0.073`, so that the probability
that this customer will churn is only `7%`. It’s less than 50%, so we will not send this customer
a promotional mail.

We can try to score another client:

In [68]:
customer = {
  'gender': 'female',
  'seniorcitizen': 1,
  'partner': 'no',
  'dependents': 'no',
  'phoneservice': 'yes',
  'multiplelines': 'yes',
  'internetservice': 'fiber_optic',
  'onlinesecurity': 'no',
  'onlinebackup': 'no',
  'deviceprotection': 'no',
  'techsupport': 'no',
  'streamingtv': 'yes',
  'streamingmovies': 'no',
  'contract': 'month-to-month',
  'paperlessbilling': 'yes',
  'paymentmethod': 'electronic_check',
  'tenure': 1,
  'monthlycharges': 85.7,
  'totalcharges': 85.7
}

x_test = dv.transform([customer])
# select the probability of churn
model.predict_proba(x_test)[0, 1]

0.8321656556545182

The output of the model is `83%` likelihood of churn, so we should send this client a
promotional mail in the hope of retaining them.

Let's combine both customers and predict its churn probability.

In [69]:
customer1 = {
  'customerid': '8879-zkjof',
  'gender': 'female',
  'seniorcitizen': 0,
  'partner': 'no',
  'dependents': 'no',
  'tenure': 41,
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'dsl',
  'onlinesecurity': 'yes',
  'onlinebackup': 'no',
  'deviceprotection': 'yes',
  'techsupport': 'yes',
  'streamingtv': 'yes',
  'streamingmovies': 'yes',
  'contract': 'one_year',
  'paperlessbilling': 'yes',
  'paymentmethod': 'bank_transfer_(automatic)',
  'monthlycharges': 79.85,
  'totalcharges': 3320.75,
}
customer2 = {
  'gender': 'female',
  'seniorcitizen': 1,
  'partner': 'no',
  'dependents': 'no',
  'phoneservice': 'yes',
  'multiplelines': 'yes',
  'internetservice': 'fiber_optic',
  'onlinesecurity': 'no',
  'onlinebackup': 'no',
  'deviceprotection': 'no',
  'techsupport': 'no',
  'streamingtv': 'yes',
  'streamingmovies': 'no',
  'contract': 'month-to-month',
  'paperlessbilling': 'yes',
  'paymentmethod': 'electronic_check',
  'tenure': 1,
  'monthlycharges': 85.7,
  'totalcharges': 85.7
}

# combine both customers
customer_data = [customer1, customer2]

x_test = dv.transform(customer_data)
# select the probability of churn
pred = model.predict_proba(x_test)

In [74]:
pred

array([[0.92667889, 0.07332111],
       [0.16783434, 0.83216566]])

In [70]:
# select the first customer's probability of churn
pred[0, 1]

0.07332111085213508

In [71]:
# select the second customer's probability of churn
pred[1, 1]

0.8321656556545182

In [72]:
# select the first customer's probability of staying
pred[0, 0]

0.9266788891478649

In [73]:
# select the second customer's probability of staying
pred[1, 0]

0.16783434434548183

So, we’ve built intuition on how logistic regression works, how to train it with
Scikit-learn, and how to apply it to new data.